In [1]:
import collections
import requests
import json
import re
import os
import sys

In [2]:
with open('../../data/speeches_20.jsonl', 'r', encoding = 'utf8') as fp:
    data = list(fp)
speeches = []
for line in data:
    speeches.append(json.loads(line))

In [3]:
def groupSpeechesByParty(speeches):
    groupedSpeeches = {}
    for speech in speeches:
        tpo = speech['party']
        if tpo in groupedSpeeches:
            groupedSpeeches[tpo].append(speech)
        else:
            groupedSpeeches[tpo] = [speech]
    return groupedSpeeches

In [4]:
def clean_partyname(speeches):
    for speech in speeches:
        if 'BÜNDNIS\xa090/DIE GRÜNEN' in speech['party']:
            speech.update({'party': 'BÜNDNIS 90/DIE GRÜNEN'})
clean_partyname(speeches)
#print(speeches[0])
#print(speech['party'])

groupbyparty = groupSpeechesByParty(speeches)
print(f'alle Reden: {len(speeches)}') # Anzahl aller Reden
for ele in groupbyparty:
    print(len(groupbyparty[ele]))
# Überprüfung mit bspw.: len(groupbyparty['SPD'])

alle Reden: 2777
698
607
457
395
353
241
26


In [5]:
# filter: Kommentare extrahieren
def extract_comments(speeches):
    regex = "{(.*?)}" # hier die genaue Definition der Kommentare vom Parser <<< >
    comments = []
    modified_speeches = []
    for speech in speeches:
        text = speech['text']
        matches = re.findall(regex,text)
        speech['comments'] = matches
        modified_speeches.append(speech)
    return modified_speeches
# Funktionsaufruf: speeches_comments = extract_comments(speeches)
speeches_comments = extract_comments(speeches)
# Funktionsausgabe: print(speeches_comments[0]['comments'])
#                   print(speeches_comments[0]['party'])

In [6]:
# filter: comments per party with contain words and not contain words
def filter_with_complex_non_words(search_terms,non_words,speeches):
    filtered_speeches = []
    for speech in speeches:
        for comment in speech['comments']:
            if all([term in comment for term in search_terms]) and not any([term in comment for term in non_words]):
                filtered_speeches.append(speech)
                break
    return filtered_speeches
# Funktionsaufruf:  specific_and_non_search = filter_with_complex_non_words(['Beifall','CDU'],['Zuruf'],speeches_comments)
# Funktionsausgabe: print(specific_and_non_search[0])                               enthält AND enthält nicht
#                   print(len(specific_and_non_search))

In [7]:
specific_and_non_search = filter_with_complex_non_words(['DIE LINKE'],[],speeches_comments)
print(f'Anzahl Kommentare mit Beteiligung von Die Linke:             {len(specific_and_non_search)}')
specific_and_non_search = filter_with_complex_non_words(['SPD'],[],speeches_comments)
print(f'Anzahl Kommentare mit Beteiligung von SPD:                   {len(specific_and_non_search)}')
specific_and_non_search = filter_with_complex_non_words(['BÜNDNIS 90/DIE GRÜNEN'],[],speeches_comments)
print(f'Anzahl Kommentare mit Beteiligung von BÜNDNIS 90/DIE GRÜNEN: {len(specific_and_non_search)}')
specific_and_non_search = filter_with_complex_non_words(['CDU/CSU'],[],speeches_comments)
print(f'Anzahl Kommentare mit Beteiligung von CDU/CSU:               {len(specific_and_non_search)}')
specific_and_non_search = filter_with_complex_non_words(['FDP'],[],speeches_comments)
print(f'Anzahl Kommentare mit Beteiligung von FDP:                   {len(specific_and_non_search)}')
specific_and_non_search = filter_with_complex_non_words(['AfD'],[],speeches_comments)
print(f'Anzahl Kommentare mit Beteiligung von AfD:                   {len(specific_and_non_search)}')
specific_and_non_search = filter_with_complex_non_words(['Fraktionslos'],[],speeches_comments)
print(f'Anzahl Kommentare mit Beteiligung von Fraktionslos:          {len(specific_and_non_search)}')

Anzahl Kommentare mit Beteiligung von Die Linke:             482
Anzahl Kommentare mit Beteiligung von SPD:                   2326
Anzahl Kommentare mit Beteiligung von BÜNDNIS 90/DIE GRÜNEN: 1635
Anzahl Kommentare mit Beteiligung von CDU/CSU:               1718
Anzahl Kommentare mit Beteiligung von FDP:                   2215
Anzahl Kommentare mit Beteiligung von AfD:                   1320
Anzahl Kommentare mit Beteiligung von Fraktionslos:          0
